In [1]:
# #This code is for adaptive GPU usage
# import keras.backend as K
# cfg = K.tf.ConfigProto()
# cfg.gpu_options.allow_growth = True
# K.set_session(K.tf.Session(config=cfg))

Using TensorFlow backend.


In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import nltk
import random
from google.colab import drive
import numpy as np
import pandas as pd
import keras.layers as L
import keras.models as M
from keras.optimizers import Adam, Adamax, Nadam, Adagrad
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier

Using TensorFlow backend.


In [2]:
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
!ls 'gdrive/My Drive/Microsoft AI'

 bert-master	      Mark_II.ipynb		      q_pairs.txt
'Finalised Version'  'Measures of Similarity.ipynb'   Results
 Junk		      pairs.txt			      test_pairs.txt


# Data Loading

In [0]:
d = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_TrainFeatures.csv",sep=',')

In [0]:
tr_data = d[d['label'] == 1].reset_index(drop=True)
fl_data = d[d['label'] == 0].reset_index(drop=True)

In [6]:
tr_data.shape, fl_data.shape

((524177, 18), (4717593, 18))

In [0]:
tr_ind = random.sample(range(0,tr_data.shape[0]),tr_data.shape[0])
fl_ind = random.sample(range(0,fl_data.shape[0]),tr_data.shape[0]*2)

In [0]:
data = pd.concat([tr_data.iloc[tr_ind],fl_data.iloc[fl_ind]],axis=0).sample(frac=1).reset_index( drop = True )

In [9]:
features = ['Cosine','Jaccard','BM25','Tfidf','Rouge P','Rouge R','BLEU 1','BLEU 2','DSSM','CDSSM',
                   'q_word_count','q_char_count','q_avg_word','r_word_count','r_char_count','r_avg_word']

split = int(10*((data.shape[0]*0.7)//10))
print(split)

1100770


In [0]:
y = data['label']
x = data[features]

x_train, x_test = x.values[:split,:], x.values[split:,:]
y_train, y_test = y.values[:split], y.values[split:]

In [11]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1100770, 16), (1100770,), (471761, 16), (471761,))

In [12]:
d.head()

,qid,Cosine,Jaccard,BM25,Tfidf,Rouge P,Rouge R,BLEU 1,BLEU 2,DSSM,CDSSM,label,q_word_count,q_char_count,q_avg_word,r_word_count,r_char_count,r_avg_word
0,131,0.280419,0.035714,0.000000,0.061623,1.0,0.035714,8.533048e-17,8.533048e-17,0.452772,0.346784,0,3,13,11.0,48,315,7.052632
1,131,0.000000,0.000000,10.204230,0.031652,0.0,0.000000,0.000000e+00,0.000000e+00,0.506993,0.182843,0,3,13,11.0,25,146,5.545455
2,131,0.116886,0.066667,0.000000,0.046495,1.0,0.066667,4.139938e-08,4.139938e-08,0.184675,0.336138,0,3,13,11.0,24,163,7.777778
3,131,0.374302,0.062500,7.216898,0.062919,1.0,0.062500,1.522998e-08,1.522998e-08,0.481602,0.227964,0,3,13,11.0,22,138,6.157895
4,131,0.284388,0.047619,10.952685,0.062085,1.0,0.047619,3.775135e-11,3.775135e-11,0.418071,0.266236,0,3,13,11.0,32,208,7.080000


In [0]:
# import seaborn as sns
# corr = data.corr()
# sns.heatmap(corr, 
#             xticklabels=x.colums.values,
#             yticklabels=y)

In [0]:
# import matplotlib.pyplot as plt
# def plot_corr(df,size=10):
#     '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

#     Input:
#         df: pandas DataFrame
#         size: vertical and horizontal size of the plot'''

#     corr = df.corr()
#     fig, ax = plt.subplots(figsize=(size, size))
#     ax.matshow(corr)
#     plt.xticks(range(len(x)), x.columns);
#     plt.yticks(range(len(y)), y.columns);

In [0]:
# plot_corr(data[:100])

# Tree Based Classifier Models

In [0]:
hyperparams = {
    "trees":{
        "criterions" : ["gini","entropy"],
        "max_features": ["log2","sqrt"], 
        "n_estimators": [300,500]}, 
    "gbc":{
        "loss" : ["deviance","exponential"],
        "learning_rate" : [0.01,0.1,1.0],
        "n_estimators" : [40,80,120],
        "max_features" : ["log2","sqrt"],
    "nn":{
        "learning_rate" : [0.1,0.01,0.05,0.001,0.005],
        "optim" : ["Adam","Adamax","Adagrad","Nadam"], 
        "activation": ["tanh","sigmoid","relu","linear"]}
}

# results = {}
# results['RFC'] = {}

# for cr in hyperparams["trees"]["criterions"]:
#     results['RFC'][str(cr)] = {}
#     for dep in hyperparams["trees"]["max_depth"]:
#         results['RFC'][str(cr)][str(dep)] = {}
#         for feat in hyperparams["trees"]["max_features"]:
#             results['RFC'][str(cr)][str(dep)][str(feat)] = {}
#             for est in hyperparams["trees"]["n_estimators"]:
#                 results['RFC'][str(cr)][str(dep)][str(feat)][str(est)] = {}
#                 print("Hyperparameters used")
#                 print("Criterion : ",cr)
#                 print("Maximum Depth : ",dep)
#                 print("Maximum Features : ",feat)
#                 print("Number of Estimators : ",est)
#                 model = RandomForestClassifier(n_estimators=est,
#                                               criterion=cr,
#                                               max_features=feat,
#                                               verbose=3,
#                                               )
                
#                 model.fit(x_train,y_train)
#                 y_pred = model.predict(x_test)
#                 imp = model.feature_importances_
#                 score = model.score(x_test,y_test)
#                 print(score)
#                 results['RFC'][str(cr)][str(dep)][str(feat)][str(est)]["score"] = score
                
#                 f = pd.DataFrame(np.column_stack((features,imp)))
#                 f.columns = ['feature','importance']
#                 print(f.sort_values(by=['importance'],ascending=False))
#                 results['RFC'][str(cr)][str(dep)][str(feat)][str(est)]["f_imp"] = f

results['ETC'] = {}

for cr in hyperparams["trees"]["criterions"]:
    results['ETC'][str(cr)] = {}
    for dep in hyperparams["trees"]["max_depth"]:
        results['ETC'][str(cr)][str(dep)] = {}
        for feat in hyperparams["trees"]["max_features"]:
            results['ETC'][str(cr)][str(dep)][str(feat)] = {}
            for est in hyperparams["trees"]["n_estimators"]:
                results['ETC'][str(cr)][str(dep)][str(feat)][str(est)] = {}
                print("Hyperparameters used")
                print("Criterion : ",cr)
                print("Maximum Depth : ",dep)
                print("Maximum Features : ",feat)
                print("Number of Estimators : ",est)
                model = ExtraTreesClassifier(n_estimators=est,
                                              criterion=cr,
                                              max_features=feat,
                                              verbose=3,
                                              warm_start=True,
                                              )
                
                model.fit(x_train,y_train)
                y_pred = model.predict(x_test)
                imp = model.feature_importances_
                score = model.score(x_test,y_test)
                print(score)
                results['ETC'][str(cr)][str(dep)][str(feat)][str(est)]["score"] = score
                
                f = pd.DataFrame(np.column_stack((features,imp)))
                f.columns = ['feature','importance']
                print(f.sort_values(by=['importance'],ascending=False))
                results['ETC'][str(cr)][str(dep)][str(feat)][str(est)]["f_imp"] = f

# results['GBC'] = {}

# for ls in hyperparams["gbc"]["loss"]:
#     results['GBC'][str(ls)] = {}
#     for lr in hyperparams["gbc"]["learning_rate"]:
#         results['GBC'][str(ls)][str(lr)] = {}
#         for est in hyperparams["gbc"]["n_estimators"]:
#             results['GBC'][str(ls)][str(lr)][str(est)] = {}
#             for feat in hyperparams["gbc"]["max_features"]:
#                 results['GBC'][str(ls)][str(lr)][str(est)][str(feat)] = {}
#                 for dep in hyperparams["gbc"]["max_depth"]:
#                     results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)] = {}
#                     print("Hyperparameters used")
#                     print("Loss : ",ls)
#                     print("Learning rate : ",lr)
#                     print("Number of Estimators : ",est)
#                     print("Maximum Features : ",feat)
#                     print("Maximum Depth : ",dep)
#                     model = GradientBoostingClassifier(loss=ls,
#                                                        learning_rate=lr,
#                                                        n_estimators=est,
#                                                        max_features=feat,
#                                                        verbose=3)

#                     model.fit(x_train,y_train)
#                     y_pred = model.predict(x_test)
#                     imp = model.feature_importances_
#                     score = model.score(x_test,y_test)
#                     print(score)
#                     results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)]["score"] = score

#                     f = pd.DataFrame(np.column_stack((features,imp)))
#                     f.columns = ['feature','importance']
#                     print(f.sort_values(by=['importance'],ascending=False))
#                     results['GBC'][str(ls)][str(lr)][str(est)][str(feat)][str(dep)]["f_imp"] = f



Hyperparameters used
Criterion :  gini
Maximum Depth :  3
Maximum Features :  log2
Number of Estimators :  20
building tree 1 of 20


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


building tree 2 of 20


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.6s remaining:    0.0s


building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.1s remaining:    0.0s


building tree 2 of 80


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   13.1s remaining:    0.0s


building tree 3 of 80
building tree 4 of 80
building tree 5 of 80
building tree 6 of 80
building tree 7 of 80
building tree 8 of 80
building tree 9 of 80
building tree 10 of 80
building tree 11 of 80
building tree 12 of 80
building tree 13 of 80
building tree 14 of 80
building tree 15 of 80
building tree 16 of 80
building tree 17 of 80
building tree 18 of 80
building tree 19 of 80
building tree 20 of 80
building tree 21 of 80
building tree 22 of 80
building tree 23 of 80
building tree 24 of 80
building tree 25 of 80
building tree 26 of 80
building tree 27 of 80
building tree 28 of 80
building tree 29 of 80
building tree 30 of 80
building tree 31 of 80
building tree 32 of 80
building tree 33 of 80
building tree 34 of 80
building tree 35 of 80
building tree 36 of 80
building tree 37 of 80
building tree 38 of 80
building tree 39 of 80
building tree 40 of 80
building tree 41 of 80
building tree 42 of 80
building tree 43 of 80
building tree 44 of 80
building tree 45 of 80
building tree 46 o

KeyboardInterrupt: 

In [0]:
# tree models tabulation
columns = ["model type","criterion","learning rate","max features","max depth","# of estimators","score"]
l = []
for mdl in results.keys():
    if(mdl != "GBC"):
        for crt in results[mdl].keys():
            for mfeat in results[mdl][crt].keys():
                for mdep in results[mdl][crt][mfeat].keys():
                    for nest in results[mdl][crt][mfeat][mdep].keys():
                        score = results[mdl][crt][mfeat][mdep][nest]["score"]
                        l.append([mdl,crt,None,mfeat,mdep,nest,score])
    else:
        for crt in results[mdl].keys():
            for lr in results[mdl][crt].keys():
                for nest in results[mdl][crt][lr].keys():
                    for mfeat in results[mdl][crt][lr][nest].keys():
                        for mdep in results[mdl][crt][lr][nest][mfeat].keys():
                            score = results[mdl][crt][lr][nest][mfeat][mdep]["score"]
                            l.append([mdl,crt,lr,mfeat,mdep,nest,score])

res = pd.DataFrame(l,columns=columns)
res = res.sort_values(by=['score'],ascending=False).reset_index(drop=True)

res.to_csv("gdrive/My Drive/Microsoft AI/Finalised Version/ETree_Hyper.csv",sep=',',index=None)

rs=pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/ETree_Hyper.csv")

rs

KeyError: 'score'

In [0]:
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import SVC
# from sklearn.grid_search import GridSearchCV
# from sklearn.metrics import f1_score

# model_to_set = OneVsRestClassifier(SVC(kernel="poly"))

# parameters = {
#     "estimator__C": [1,2,4,8],
#     "estimator__kernel": ["poly","rbf"],
#     "estimator__degree":[1, 2, 3, 4],
# }

# model_tunning = GridSearchCV(model_to_set, param_grid=parameters,
#                              score_func=f1_score)

# model_tunning.fit(x_train,y_train)

# print (model_tunning.best_score_)
# print (model_tunning.best_params_)

In [0]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
model1 = OneVsRestClassifier(SVC(kernel = 'rbf')).fit(x_train, y_train)
y_pred = model1.predict(x_test)
#imp = model1.feature_importances_
score = model1.score(x_test,y_test)
print(score)

#pd.DataFrame(np.column_stack((features,imp)))

In [0]:
hyperparams = {
    "trees":{
        "criterion" : ["gini","entropy"],
        "max_depth" : [3,4,5,6,7,8,9,10],
        "max_features": ["log2","sqrt","auto"], 
        "n_estimators": [20,40,60,80,100]}, 
    "gbc":{
        "loss" : ["deviance","exponential"],
        "learning_rate" : [0.1,0.5,0.8,1.0],
        "n_estimators" : [20,40,60,80,100],
        "max_features" : ["log2","sqrt","auto"],
        "max_depth" : [3,4,5,6,7,8,9,10]},
    "nn":{
        "learning_rate" : [0.1,0.01,0.05,0.001,0.005],
        "optim" : ["Adam","Adamax","Adagrad","Nadam"], 
        "activation": ["tanh","sigmoid","relu","linear"]}
}

In [0]:
cr = "gini"
ls = 'exponential'
lr = 0.1
est = 100
feat = "log2"
dep = 7

In [15]:
RF = RandomForestClassifier(n_estimators=est,criterion=cr,max_features=feat,max_depth=dep,verbose=3)#,class_weight="balanced_subsample")
RF.fit(x_train,y_train)

y_pred = RF.predict(x_test)
imp = RF.feature_importances_
score = RF.score(x_test,y_test)
print(score)

pd.DataFrame(np.column_stack((features,imp)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s remaining:    0.0s


building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    9.2s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  7.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


0.6826316715455496


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.0s finished


,0,1
0,Cosine,0.06344258591645387
1,Jaccard,0.08593274477955601
2,BM25,0.04202367892049407
3,Tfidf,0.006156089459379932
4,Rouge P,0.35577436027499887
5,Rouge R,0.06562902340840678
6,BLEU 1,0.009012871727757298
7,BLEU 2,0.00959041317509271
8,DSSM,0.14892862785129735
9,CDSSM,0.17845389356954858


In [0]:
#model type,criterion,learning rate,max features,max depth,# of estimators,score
#GBC,exponential,0.1,log2,7,100,0.900060606060606
#RFC,gini,,10,auto,80,0.6947878787878788

GB = GradientBoostingClassifier(loss=ls, learning_rate=lr,n_estimators=est,max_features=feat,max_depth=dep,verbose=3)

GB.fit(x_train,y_train)

y_pred = GB.predict(x_test)
imp = GB.feature_importances_
score = GB.score(x_test,y_test)
print(score)

pd.DataFrame(np.column_stack((features,imp)))

      Iter       Train Loss   Remaining Time 
         1           0.9328           27.77m
         2           0.9247           28.05m
         3           0.9175           27.01m
         4           0.9106           27.55m
         5           0.9049           27.03m
         6           0.8999           26.94m
         7           0.8958           26.58m
         8           0.8923           26.12m
         9           0.8893           24.81m
        10           0.8866           23.44m
        11           0.8843           22.14m
        12           0.8823           20.92m
        13           0.8806           19.88m
        14           0.8791           18.69m
        15           0.8778           18.69m
        16           0.8767           18.73m


In [0]:
from sklearn.multiclass import OneVsRestClassifier
from pylightgbm.models import GBMRegressor
from sklearn.ensemble import ExtraTreesClassifier

ET = ExtraTreesClassifier(n_estimators=450, criterion='gini', verbose=0,n_jobs=-1, max_features='auto',min_samples_split=3, min_samples_leaf=20 )#, warm_start = True)


ET.fit(x_train,y_train)

y_pred = ET.predict(x_test)
imp = ET.feature_importances_
score = ET.score(x_test,y_test)
print(score)

pd.DataFrame(np.column_stack((features,imp)))

In [0]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,
        random_state=0).fit(x_train, y_train)

score = gpc.score(x_test, y_test) 
print(score)

y_pred = ET.predict(x_test)
imp = ET.feature_importances_

pd.DataFrame(np.column_stack((features,imp)))

### Max Voting

In [0]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('lr', GB), ('dt', ET)], voting='hard')
model.fit(x_train,y_train)
model.score(x_test,y_test)

### Stacking

In [0]:
def Stacking(model,train,y,test,n_fold):
       folds=StratifiedKFold(n_splits=n_fold,random_state=1)
       test_pred=np.empty((test.shape[0],1),float)
       train_pred=np.empty((0,1),float)
       for train_indices,val_indices in folds.split(train,y.values):
          x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
          y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

          model.fit(X=x_train,y=y_train)
          train_pred=np.append(train_pred,model.predict(x_val))
          test_pred=np.append(test_pred,model.predict(test))
        return test_pred.reshape(-1,1),train_pred


In [0]:
model1 = ET

test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=x_train,test=x_test,y=y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

In [0]:
model2 = GB

test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=x_train,test=x_test,y=y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [0]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test, y_test)

## Neural Network

results['nn'] = {}
for optim in hyperparams['nn']['optim']:
    results['nn'][str(optim)] = {}
    for lr in hyperparams['nn']['learning_rate']:
        results['nn'][str(optim)][str(lr)] = {}
        for act in hyperparams['nn']['activation']:
            print("Hyperparameter used")
            print("Optimizer : ",optim)
            print("Learning rate : ",lr)
            print("Activation : ",act)
            results['nn'][str(optim)][str(lr)][str(act)] = {}
            inp = L.Input(shape=(len(features),))
            x = L.Dense(512, activation=act)(inp)
            x = L.Dense(256, activation=act)(x)
            x = L.Dense(128, activation=act)(x)
            x = L.Dense(128, activation=act)(x)
            x = L.Dense(64, activation=act)(x)
            out = L.Dense(1, activation="sigmoid")(x)
            
            model = M.Model(inputs=[inp],outputs=[out])
            model.summary()
            if optim == "Adam":
                o = Adam(lr)
            elif optim == "Adagrad":
                o = Adagrad(lr)
            elif optim == "Adamax":
                o = Adamax(lr)
            elif optim == "Nadam":
                o = Nadam(lr)
            model.compile(o,loss='binary_crossentropy',metrics=['accuracy'])
            
            model.fit(x_train,y_train,epochs=10)
            score = model.evaluate(x_test,y_test)
            y_pred = model.predict(x_test)
            
            results['nn'][str(optim)][str(lr)][str(act)]["score"] = score

# tree models tabulation
columns = ["optimizer","learning rate","activation","score"]
l = []
for mdl in results.keys():
    if(mdl == "nn"):
        for optim in results[mdl].keys():
            for lr in results[mdl][optim].keys():
                for act in results[mdl][optim][lr].keys():
                    score = results[mdl][optim][lr][act]["score"]
                    l.append([optim,lr,act,score])

res = pd.DataFrame(l,columns=columns)
res["loss"] = [s[0] for s in res["score"]]
res["score"] = [s[1] for s in res["score"]]

res = res.sort_values(by=['score'],ascending=False).reset_index(drop=True)

res

res.to_csv("NN_Hyper.csv",sep=',',index=None)

# optimizer,learning rate,activation,score,loss
# Adamax,0.1,tanh,0.8999999999711008,0.37295132941910714

In [0]:
inp = L.Input(shape=(len(features),))
x = L.Dense(128, activation='sigmoid')(inp)
x = L.Dense(256, activation='sigmoid')(x)
x = L.Dense(64, activation='sigmoid')(x)
out = L.Dense(1, activation='sigmoid')(x)

model = M.Model(inputs=[inp],outputs=[out])
model.summary()

o = Adamax(0.1)
model.compile(optimizer=o,loss='binary_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=10,batch_size=128,validation_data=(x_test,y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 12)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1664      
_________________________________________________________________
dense_10 (Dense)             (None, 256)               33024     
_________________________________________________________________
dense_11 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 51,201
Trainable params: 51,201
Non-trainable params: 0
_________________________________________________________________
Train on 1467690 samples, validate on 629018 samples
Epoch 1/10
1467690/1467690 [==============================] - 60s 41us/step - loss: 0

# Evaluation Code

In [0]:
d2 = pd.read_csv("gdrive/My Drive/Microsoft AI/Finalised Version/All_TestFeatures.csv",sep=',')

x_test_actual = d2[features]

In [0]:
len(d2)

104170

In [0]:
#X_test = x_test_actual.fillna(x_train.mean())
d2.head(20)

,qid,Cosine,Jaccard,BM25,Tfidf,Rouge P,Rouge R,BLEU 1,BLEU 2,DSSM,CDSSM,q_word_count,q_char_count,q_avg_word,r_word_count,r_char_count,r_avg_word
0,1135787,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.314036,0.263362,1,1,1.0,1,1,1.000000
1,1135787,0.000000,0.130435,35.142888,0.000000,0.600000,0.142857,4.042768e-03,4.042768e-03,0.760360,0.549321,5,30,5.2,36,210,5.833333
2,1135787,0.000000,0.130435,31.517606,0.000000,0.600000,0.142857,7.366404e-03,7.366404e-03,0.677732,0.546766,5,30,5.2,32,187,5.777778
3,1135787,0.000000,0.142857,33.750318,0.000000,1.000000,0.142857,3.043248e-05,3.043248e-05,0.589114,0.464406,5,30,5.2,71,337,4.684211
4,1135787,0.000000,0.097561,25.981487,0.000000,0.800000,0.100000,1.798939e-04,1.798939e-04,0.398033,0.260062,5,30,5.2,59,370,6.638298
5,1135787,0.000000,0.095238,30.543686,0.000000,0.800000,0.097561,2.208618e-06,2.208618e-06,0.428608,0.201067,5,30,5.2,81,479,5.782609
6,1135787,0.000000,0.080000,8.438989,0.000000,0.400000,0.090909,1.825411e-07,1.825411e-07,0.232884,0.245269,5,30,5.2,119,342,2.871795
7,1135787,0.000000,0.058824,19.537679,0.000000,0.400000,0.064516,6.646229e-04,6.646229e-04,0.485853,0.332196,5,30,5.2,50,283,6.324324
8,1135787,0.000000,0.081081,31.756525,0.000000,0.600000,0.085714,3.327096e-05,3.327096e-05,0.609475,0.484145,5,30,5.2,69,356,5.333333
9,1135787,0.000000,0.130435,35.362967,0.000000,0.600000,0.142857,4.042768e-03,4.042768e-03,0.760360,0.549321,5,30,5.2,35,209,5.833333


In [0]:
'''
IF Neural Net Model

'''
probs = model.predict(x_test_actual).ravel()

p = []
for i in range(0,d2.shape[0],10):
    l = []
    v = probs[i:i+10]
    l.append(d2['qid'][i])
    l.extend(v)
    p.append(l)

result = pd.DataFrame(p)

result.to_csv("answer.tsv",sep='\t',header=None,index=False)

In [0]:
'''
IF Tree Model

'''
probs = model1.predict_proba(x_test_actual)[:,1]

p = []
for i in range(0,d2.shape[0],10):
    l = []
    v = probs[i:i+10].T
    l.append(d2['qid'][i])
    l.extend(v)
    p.append(l)

result = pd.DataFrame(p)

result.to_csv("answer.tsv",sep='\t',header=None,index=False)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.0s finished


In [0]:
x_test_actual.head(30)

In [0]:
# x_test_new = pd.concat([x_test_actual,  pd.DataFrame({'DSSM':0, 'CDSSM':0}, index=[0]), x_test_actual.loc[1:]], ignore_index=True).copy(deep=True)
# x_test_new.head()

,BLEU 1,BLEU 2,BM25,CDSSM,Cosine,DSSM,Jaccard,Rouge P,Rouge R,Tfidf,q_avg_word,q_char_count,q_word_count,r_avg_word,r_char_count,r_word_count
0,0.000000,0.000000,0.000000,0.549321,0.0,0.760360,0.000000,0.0,0.000000,0.0,1.0,1.0,1.0,1.000000,1.0,1.0
1,0.004043,0.004043,35.142888,0.546766,0.0,0.677732,0.130435,0.6,0.142857,0.0,5.2,30.0,5.0,5.833333,210.0,36.0
2,0.007366,0.007366,31.517606,0.464406,0.0,0.589114,0.130435,0.6,0.142857,0.0,5.2,30.0,5.0,5.777778,187.0,32.0
3,0.000030,0.000030,33.750318,0.260062,0.0,0.398033,0.142857,1.0,0.142857,0.0,5.2,30.0,5.0,4.684211,337.0,71.0
4,0.000180,0.000180,25.981487,0.201067,0.0,0.428608,0.097561,0.8,0.100000,0.0,5.2,30.0,5.0,6.638298,370.0,59.0
